In [1]:
import pandas as pd
import json
import os
import numpy as np

import sentiwordnet
import wndomains
import geowordnet

import gensim
import gensim.corpora as corpora
from gensim.corpora import Dictionary
from gensim.utils import simple_preprocess
from gensim.models import CoherenceModel
from gensim.models.ldamodel import LdaModel

from nltk.corpus import stopwords
from nltk.tokenize import sent_tokenize
from nltk.stem import WordNetLemmatizer
#import spacy
import nltk
nltk.download('stopwords')

from nltk.corpus import wordnet as wn

# spacy for lemmatization
import spacy
#nlp = spacy.load('en', disable=['parser', 'ner'])
nlp = spacy.load('en')

C:\Users\TUSHAR\Anaconda3\lib\site-packages\gensim\utils.py:1209: UserWarning: detected Windows; aliasing chunkize to chunkize_serial
  warnings.warn("detected Windows; aliasing chunkize to chunkize_serial")


[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\TUSHAR\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [2]:
# tokenize
def tokenize_words(text):
    token = []
    count=0
    lemmat_tokens=[]
    wordnet_lemmatizer = WordNetLemmatizer()
    total = len(text)
    #print((total))    
    try:        
        for i in range(total):
            count=count+1
            if(text[i] == '' or pd.isnull(text[i])):
                #print("empty str")
                num=9
            else:
                swtoken=[]
                swrtoken=[]
                swtoken=gensim.utils.simple_preprocess(text[i],deacc=True)
                token.append(swtoken)
                    
    except:
        print("text = ",text[i])
    
    return token 

In [3]:
path = 'C:\\Users\\TUSHAR\\Desktop\\unis_MSD\\capstone project\\notion\\dataset\\R4'
os.chdir(path)

data_df=pd.read_csv('article_content.csv', encoding='utf-8')
#data_df=pd.read_csv('20_article_content.csv', encoding='utf-8')
summary_text=list(data_df['content'])

from nltk.corpus import stopwords
stopWords = stopwords.words('english')
#stopWords = list(stopwords.words('english'))
stopWords.extend(["commission","royal","said","australia","let","told","also","ms","com","www","https","au","us","would","mailto","subject","body","says","orr","acbf","mr"]) 

#tokens = tokenize_words(summary_text)
data_words = tokenize_words(summary_text)

big_corpus=''
for article in summary_text:
    big_corpus=big_corpus+np.str(article)

In [4]:
def spacyNER(text):
    nlp = spacy.load('en')
    nerList=[]
    doc=nlp(text)
    for ent in doc.ents:
        nerEntry=ent.text+'_'+ent.label_
        nerList.append(nerEntry)
    print(nerList)
    return nerList

big_corpus1='Apple is a big company in China'
for article in summary_text[:10]:
    big_corpus=big_corpus+np.str(article)

#nerList = spacyNER(big_corpus1)

In [5]:
#print(nerList)

In [6]:
# Build the bigram and trigram models
bigram = gensim.models.Phrases(data_words, min_count=1, threshold=2) # higher threshold fewer phrases.
trigram = gensim.models.Phrases(bigram[data_words], threshold=2)  

# Faster way to get a sentence clubbed as a trigram/bigram
bigram_mod = gensim.models.phrases.Phraser(bigram)
trigram_mod = gensim.models.phrases.Phraser(trigram)

# See trigram example
#print(trigram_mod[bigram_mod[data_words[0]]])

C:\Users\TUSHAR\Anaconda3\lib\site-packages\gensim\models\phrases.py:598: UserWarning: For a faster implementation, use the gensim.models.phrases.Phraser class
  warnings.warn("For a faster implementation, use the gensim.models.phrases.Phraser class")


In [7]:
# Define functions for stopwords, bigrams, trigrams and lemmatization
def remove_stopwords(texts):
    return [[word for word in simple_preprocess(str(doc)) if word not in stopWords] for doc in texts]

def make_bigrams(texts):
    return [bigram_mod[doc] for doc in texts]

def make_trigrams(texts):
    return [trigram_mod[bigram_mod[doc]] for doc in texts]

def lemmatization(texts, allowed_postags=['NOUN', 'ADJ', 'VERB', 'ADV']):
    """https://spacy.io/api/annotation"""
    texts_out = []
    for sent in texts:
        doc = nlp(" ".join(sent)) 
        texts_out.append([token.lemma_ for token in doc if token.pos_ in allowed_postags])
    return texts_out

def spacyNER(text):
    nlp = spacy.load('en_core_web_sm')
    nerList=[]
    doc=nlp(text)
    for ent in doc.ents:
        nerEntry=ent.text+'_'+ent.label_
        nerList.append(nerEntry)
    return nerList

In [8]:
# Remove Stop Words
data_words_nostops = remove_stopwords(data_words)

# Form Bigrams
data_words_bigrams = make_bigrams(data_words_nostops)
data_words_trigrams = make_trigrams(data_words_nostops)

# Initialize spacy 'en' model, keeping only tagger component (for efficiency)
# python3 -m spacy download en
#nlp = spacy.load('en', disable=['parser', 'ner'])
nlp = spacy.load('en')

# Do lemmatization keeping only noun, adj, vb, adv
data_lemmatized = lemmatization(data_words_trigrams, allowed_postags=['NOUN', 'ADJ', 'VERB', 'ADV'])

#nerList = spacyNER(big_corpus)
#print(data_lemmatized[:1])

In [9]:
#print(data_words_trigrams)

In [10]:
# Create Dictionary
id2word = corpora.Dictionary(data_lemmatized)

# Create Corpus
texts = data_lemmatized

# Term Document Frequency
corpus = [id2word.doc2bow(text) for text in texts]

# View
#print(corpus[:1])

In [8]:
#print(id2word.id2token)

In [11]:
# Build LDA model
lmodel = gensim.models.ldamodel.LdaModel(corpus=corpus,
                                           id2word=id2word,
                                           num_topics=30, 
                                           random_state=100,
                                           update_every=1,
                                           chunksize=100,
                                           passes=10,
                                           iterations=50,
                                           alpha='auto',
                                           per_word_topics=True)

In [9]:
#tokens = tokenize_words(content)


In [12]:
ltopics=[]
for i in range(lmodel.num_topics):
    topic_list_n=lmodel.print_topic(i)
    temp_token_1=topic_list_n.split('+')
    for ctoken in temp_token_1:
        temp_token_2=ctoken.split('*')
        temp_token_3 = temp_token_2[1]
        temp_token_4=temp_token_3.split('"')
        ind_topic=temp_token_4[1]
    #print(itop)
        ltopics.append(np.str(ind_topic))
    
print(ltopics)
unique_topic_list=list(set(ltopics))
print(unique_topic_list)
    


['investor', 'market', 'fall', 'stock', 'growth', 'price', 'tighten', 'today', 'sector', 'line', 'fund', 'manage', 'member', 'interest_rate', 'look', 'government', 'set', 'level', 'own', 'invest', 'funeral_plan', 'fund', 'community_benefit', 'fall', 'jone', 'funeral_insurer', 'corporate_watchdog', 'describe', 'advertising', 'scrutiny_aboriginal', 'bank', 'loan', 'banking', 'customer', 'policy', 'hear', 'call', 'sell', 'company', 'meet', 'think', 'need', 'year', 'australian', 'important', 'company', 'want', 'new', 'industry', 'china', 'funeral_insurance', 'walsh', 'family', 'work', 'indigenous_communitie', 'tracey_walsh', 'insurer', 'indigenous_consumer', 'fund', 'probe', 'bank', 'rate', 'borrower', 'continue', 'cost', 'ability', 'growth', 'increase', 'share', 'big_four', 'child', 'torres_strait', 'aboriginal', 'funeral', 'admit', 'rowena', 'targeted_indigenous', 'counsel_assisting', 'community', 'polici', 'monetary_policy', 'response', 'financial_crisis', 'limit', 'use', 'next_three', 

In [13]:
short_topic_list=unique_topic_list[:50]
print(len(unique_topic_list))

262


In [24]:
interested_doomains1=['administration','banking','commerce','economy','finance','health','humanities','insurance','money','social_science','tax']
interested_doomains2=['geography','banking','economy','finance','insurance','social_science']
interested_doomains3=['geography','law','finance','social_science']
interested_doomains4=['sociology','geography','law','social_science']
interested_doomains5=['sociology','social_science']
interested_doomains=['banking','economy','finance']
interested_topics=[]
strength_list=[]

for composite_topic in unique_topic_list:
    try:
        tlist=composite_topic.split('_')
        for topic in tlist:
            s = wn.synsets(topic)[0]
            msc = wn.synset(s.name())
            dtup=msc.domains()[:5]
            #if composite_topic== 'funeral_insurance':
                #print("funeral_insurance", dtup)
            for tup in dtup:
                domain=tup[0]
                strength=tup[1]
                if domain in interested_doomains:
                    if (composite_topic not in interested_topics and s.pos() == 'n'):
                        interested_topics.append(composite_topic)
                        strength_list.append(strength)
                        #print('topic= ',composite_topic,'domain= ',domain,'strength= ','strength= ',strength, 'pos= ',s.pos())
    except:
        print('exception for topic ',composite_topic)
            
print(interested_topics)
print(len(interested_topics))

dfn=pd.DataFrame({'topic':interested_topics,'strength':strength_list})
#dfn.columns=['topic','strength']
len_total=dfn.topic.count()
len_desired=np.int(len_total - round(len_total*.1))
print('len_desired ',len_desired)

sdf = dfn.sort_values(['strength'], ascending=False)['topic']
#df_top=sdf['topic'].tolist()
temp_topics=sdf.iloc[:np.int(len_desired)]
final_topics=temp_topics.tolist()
#print(df_top.head())
print(final_topics)

exception for topic  smh
exception for topic  indigenous_communitie
exception for topic  jone
exception for topic  clayton_utz
exception for topic  marika
exception for topic  per_cent
exception for topic  asx_index
exception for topic  prime_minister_malcolm_turnbull
exception for topic  polici
exception for topic  torres_strait_islander_people
exception for topic  tracey_walsh
exception for topic  brauer
exception for topic  sales_representatif
exception for topic  cba
exception for topic  howden
exception for topic  ceo_bryn
exception for topic  australian_securitie
exception for topic  westpac
exception for topic  heard_anz
exception for topic  lynda_edward
exception for topic  rowena
exception for topic  rba
exception for topic  tuesday_asic
exception for topic  walsh
exception for topic  theaustralian
exception for topic  financial_plann
exception for topic  torres_strait
exception for topic  nathan_boyle
exception for topic  victoria_believ
['sydney_call', 'home_loan', 'employee

In [36]:
s = wn.synsets('mortgage')[0]
print(s.name())
msc = wn.synset(s.name())
#print(msc.domains()[:5])
dict1=msc.domains()[:5]
for key in dict1:
    #dd=key.split(',')
    domain=key[0]
    print(domain, key[1])

mortgage.n.01
finance 0.00215095
exchange 0.000477929
banking 0.000434492
economy 0.000242608
enterprise 0.000193468


In [25]:
topics1=['aboriginal','islander','indigenous','mortality','breaches']
#topics=['housing','loans','decreases','consecutive','trend']
#topics2=unique_topic_list
topics2=final_topics
topics1=final_topics

#tokensl1=tokens
len_docs=len(data_lemmatized)
print(len_docs)
window=50
topic_freq=dict()
co_occur_net=dict()

for t1 in topics1:
    for t2 in topics2:
        if(t1!= t2):
            #Loop through the docs
            for i in range(len_docs):
                tokensl1=data_lemmatized[i]                
                l=len(tokensl1)
                #Loop through the tokens of a document
                for j in range(l):
                    if(tokensl1[j]==t1):
                        if(j+1+window > l-1):
                            end=l
                        else:
                            end=j+1+window
            
                        str=tokensl1[j+1:end]
                        #print(str)
                        if(t2 in str):
                            #print(t1,t2,"co-occurrence")
                            tkey=t1+':'+t2
                            if(tkey in co_occur_net.keys()):
                                co_freq=co_occur_net[tkey]
                                co_occur_net[tkey]=co_freq+1
                            else:
                                co_occur_net[tkey]=1
                                

711


In [26]:
#len_docs=len(tokens)
topic_freq=dict()

for t1 in topics1:
    for i in range(len_docs):
        tokensl1=data_lemmatized[i]
        l=len(tokensl1)
        #Loop through the tokens of a document
        for j in range(l):
            if(tokensl1[j]==t1):
                if(t1 in topic_freq.keys()):
                    topic_freq[t1]=topic_freq[t1]+1
                else:
                    topic_freq[t1]=1
                


print(topic_freq)
print("co_occur")
#print(co_occur_net)

{'tax_cut': 19, 'money': 36, 'loan': 129, 'home_loan': 45, 'customer': 123, 'banking': 152, 'mortgage': 33, 'interest_mortgage': 4, 'law_firm': 5, 'wealth_management': 39, 'cash_rate': 28, 'pay': 88, 'financial_service': 83, 'service': 75, 'business': 216, 'cost': 76, 'debt': 46, 'price': 45, 'state': 29, 'expense': 17, 'want': 51, 'property': 79, 'employee': 14, 'economy': 50, 'car': 16, 'borrower': 28, 'trust': 23, 'fund': 111, 'finance': 43, 'community_benefit': 17, 'broker': 62, 'product': 83, 'family': 61, 'amount': 28, 'trade_war': 24, 'insurance_company': 23, 'health_insurance': 12, 'insurance': 67, 'insurance_industry': 7, 'indigenous_consumer': 20, 'sydney_call': 10, 'call': 71, 'investor': 84, 'industry': 83, 'market': 157, 'lender': 58, 'stock': 50, 'financial_adviser': 24, 'adviser': 55, 'financial_crisis': 14, 'funding_cost': 10, 'ceo_bryn': 6, 'sale': 89, 'failure': 19, 'increase': 81, 'cut': 52, 'insurer': 24, 'reason': 19, 'share': 30, 'wait': 11, 'housing_market': 18, 

In [29]:
edge_weight_conet=dict()

for key,val in co_occur_net.items():
    composite_topics=key.split(':')
    source_topic=composite_topics[0]    
    if(source_topic in topic_freq.keys()):
        ind_freq=topic_freq[source_topic]
        if(ind_freq > 0):
            ew= val/np.int(ind_freq)
            if(ew>0.2):
                edge_weight_conet[key]=ew
    

In [30]:
print(edge_weight_conet)

{'tax_cut:business': 0.5789473684210527, 'tax_cut:want': 0.21052631578947367, 'tax_cut:cut': 0.42105263157894735, 'home_loan:loan': 0.2222222222222222, 'home_loan:customer': 0.2222222222222222, 'home_loan:cash_rate': 0.26666666666666666, 'home_loan:pay': 0.2222222222222222, 'home_loan:investor': 0.28888888888888886, 'home_loan:lender': 0.2222222222222222, 'interest_mortgage:broker': 1.0, 'law_firm:banking': 0.4, 'law_firm:insurance_industry': 0.4, 'law_firm:firm': 0.4, 'wealth_management:business': 0.5128205128205128, 'cash_rate:banking': 0.21428571428571427, 'cash_rate:cost': 0.21428571428571427, 'cash_rate:increase': 0.21428571428571427, 'pay:funeral_insurance': 0.22727272727272727, 'financial_service:product': 0.20481927710843373, 'cost:increase': 0.23684210526315788, 'debt:property': 0.2608695652173913, 'debt:product': 0.21739130434782608, 'price:market': 0.26666666666666666, 'expense:customer': 0.29411764705882354, 'expense:market': 0.23529411764705882, 'expense:premium': 0.294117

In [60]:
# Make CSV of bigram co-occurrence net
sourceNode=[]
targetNode=[]
edgeWeight=[]

for key,val in edge_weight_conet.items():
    composite_topics=key.split(':')
    sourceNode.append(composite_topics[0])
    targetNode.append(composite_topics[1])
    edgeWeight.append(val)
    
cooccur_df=pd.DataFrame()
cooccur_df['source']=sourceNode
cooccur_df['target']=targetNode
cooccur_df['weight']=edgeWeight

cooccur_df.to_csv('50_topics_cooccurrence_net.csv',index=False,encoding='utf-8')

In [32]:
# Now work on trigrams:

#edge_weight_conet=dict()
trigram_weight_conet=dict()
stop_words_trigram=['hear', 'business','australian','time','believe','support','public','new','fall','smh','html','make','june','child','year']

for key,val in edge_weight_conet.items():
    composite_topics=key.split(':')
    bi_source_topic=composite_topics[0]    
    bi_target_topic=composite_topics[1]
    if((bi_source_topic not in stop_words_trigram) and (bi_target_topic not in stop_words_trigram)):
        for key2,val2 in edge_weight_conet.items():
            composite_topics2=key2.split(':')
            inner_source_topic=composite_topics2[0]
            inner_target_topic=composite_topics2[1]
            #print(bi_target_topic , inner_source_topic)
            if (bi_target_topic == inner_source_topic and inner_target_topic != bi_source_topic and (inner_target_topic not in stop_words_trigram)):
                trigram_key=bi_source_topic+':'+bi_target_topic+':'+inner_target_topic
                trigram_weight=val*val2
                if(trigram_weight > .1):
                    trigram_weight_conet[trigram_key]=trigram_weight
            #if (bi_source_topic == inner_target_topic and inner_source_topic != bi_target_topic and (inner_source_topic not in stop_words_trigram)):
                #trigram_key2=inner_source_topic+':'+inner_target_topic+':'+bi_target_topic
                #trigram_weight2=val*val2
                #if(trigram_weight > .1):
                    #trigram_weight_conet[trigram_key2]=trigram_weight2
    
print(trigram_weight_conet)

{'tax_cut:cut:want': 0.10526315789473684, 'interest_mortgage:broker:industry': 0.3064516129032258, 'law_firm:insurance_industry:state': 0.11428571428571428, 'law_firm:insurance_industry:insurer': 0.11428571428571428, 'law_firm:insurance_industry:firm': 0.11428571428571428, 'law_firm:insurance_industry:produce': 0.11428571428571428, 'law_firm:insurance_industry:claim': 0.11428571428571428, 'expense:premium:pay': 0.16176470588235295, 'expense:premium:family': 0.10294117647058823, 'expense:premium:insurance_company': 0.10294117647058823, 'expense:premium:funeral_plan': 0.10294117647058823, 'expense:premium:funeral_insurance': 0.1323529411764706, 'community_benefit:insurance:funeral_insurance': 0.11237928007023705, 'community_benefit:funeral_plan:fund': 0.15441176470588236, 'community_benefit:funeral_plan:family': 0.20588235294117646, 'community_benefit:funeral_plan:funeral_insurance': 0.10294117647058823, 'insurance_company:funeral_plan:fund': 0.16304347826086957, 'insurance_company:funer

In [21]:
# Make CSV of trigram co-occurrence net
sourceNode_trig=[]
targetNode_trig=[]
edgeWeight_trig=[]

for key,val in trigram_weight_conet.items():
    composite_topics=key.split(':')
    source=composite_topics[0]+':'+composite_topics[1]
    sourceNode_trig.append(source)
    targetNode_trig.append(composite_topics[2])
    edgeWeight_trig.append(val)
    
tri_cooccur_df=pd.DataFrame()
tri_cooccur_df['source']=sourceNode_trig
tri_cooccur_df['target']=targetNode_trig
tri_cooccur_df['weight']=edgeWeight_trig

#tri_cooccur_df.to_csv('50top_lawDom_3gram__cooccurrence_net.csv',index=False,encoding='utf-8')
tri_cooccur_df.to_csv('30top_SLG_Dom_3gram__cooccurrence_net.csv',index=False,encoding='utf-8')

In [34]:
# Now work on 4grams:

#trigram_weight_conet=dict()
fourgram_weight_conet=dict()

for key3,val3 in trigram_weight_conet.items():
    composite_topics=key3.split(':')
    tri_source_topic1=composite_topics[0]
    tri_source_topic2=composite_topics[1]
    tri_target_topic=composite_topics[2]
    for key4,val4 in edge_weight_conet.items():
        composite_topics2=key4.split(':')
        inner_source_topic=composite_topics2[0]
        inner_target_topic=composite_topics2[1]
        #print(bi_target_topic , inner_source_topic)
        if (tri_target_topic == inner_source_topic and inner_target_topic != tri_source_topic1 and inner_target_topic != tri_source_topic2 and (inner_target_topic not in stop_words_trigram)):
            fourgram_key=tri_source_topic1+':'+tri_source_topic2+':'+tri_target_topic+':'+inner_target_topic
            fourgram_weight=val3*val4
            if(fourgram_weight > 0.1):
                fourgram_weight_conet[fourgram_key]=fourgram_weight
        #if (tri_source_topic1 == inner_target_topic and inner_source_topic != tri_target_topic and inner_source_topic != tri_source_topic2 and (inner_source_topic not in stop_words_trigram)):
            #fourgram_key=inner_source_topic+':'+inner_target_topic+':'+tri_source_topic2+':'+tri_target_topic
            #fourgram_weight=val3*val4
            #if(fourgram_weight > 0.1):
            #fourgram_weight_conet[fourgram_key]=fourgram_weight
    
    
print(fourgram_weight_conet)

{'sydney_call:funeral_insurance:premium:pay': 0.12767857142857145, 'scrutiny_aboriginal:community_benefit:insurance:funeral_insurance': 0.11237928007023705, 'scrutiny_aboriginal:community_benefit:funeral_plan:fund': 0.15441176470588236, 'scrutiny_aboriginal:community_benefit:funeral_plan:family': 0.20588235294117646, 'scrutiny_aboriginal:community_benefit:funeral_plan:funeral_insurance': 0.10294117647058823, 'rent:scrutiny_aboriginal:community_benefit:fund': 0.21568627450980393, 'rent:scrutiny_aboriginal:community_benefit:product': 0.11764705882352941, 'rent:scrutiny_aboriginal:community_benefit:insurance': 0.1568627450980392, 'rent:scrutiny_aboriginal:community_benefit:funeral_plan': 0.1372549019607843, 'rent:scrutiny_aboriginal:funeral_plan:fund': 0.125, 'rent:scrutiny_aboriginal:funeral_plan:family': 0.16666666666666666, 'rent:funeral_insurer:scrutiny_aboriginal:fund': 0.11904761904761904, 'rent:funeral_insurer:scrutiny_aboriginal:community_benefit': 0.11904761904761904, 'rent:funer

In [35]:
# Make CSV of fourgram co-occurrence net

sourceNode_4g=[]
targetNode_4g=[]
edgeWeight_4g=[]

for key,val in fourgram_weight_conet.items():
    composite_topics=key.split(':')
    source=composite_topics[0]+':'+composite_topics[1]+':'+composite_topics[2]
    sourceNode_4g.append(source)
    targetNode_4g.append(composite_topics[3])
    edgeWeight_4g.append(val)
    
four_cooccur_df=pd.DataFrame()
four_cooccur_df['source']=sourceNode_4g
four_cooccur_df['target']=targetNode_4g
four_cooccur_df['weight']=edgeWeight_4g

#four_cooccur_df.to_csv('30top_SLG_Dom_4gram_cooccurrence_net.csv',index=False,encoding='utf-8')
four_cooccur_df.to_csv('30top_Finance_Dom_4gram_cooccurrence_net.csv',index=False,encoding='utf-8')

In [68]:
# Try 5 gram:

#fourgram_weight_conet=dict()
fivegram_weight_conet=dict()

for key5,val5 in fourgram_weight_conet.items():
    composite_topics=key5.split(':')
    four_source_topic1=composite_topics[0]
    four_source_topic2=composite_topics[1]
    four_source_topic3=composite_topics[2]
    four_target_topic=composite_topics[3]
    for key6,val6 in edge_weight_conet.items():
        composite_topics2=key6.split(':')
        inner_source_topic=composite_topics2[0]
        inner_target_topic=composite_topics2[1]
        #print(bi_target_topic , inner_source_topic)
        if (four_target_topic == inner_source_topic and inner_target_topic != four_source_topic1 
            and inner_target_topic != four_source_topic2 and inner_target_topic != four_source_topic3
            and (inner_target_topic not in stop_words_trigram)):
            fivegram_key=four_source_topic1+':'+four_source_topic2+':'+four_source_topic3+':'+four_target_topic+':'+inner_target_topic
            fivegram_weight=val5*val6
            #if(fivegram_weight > 0.1):
            fivegram_weight_conet[fivegram_key]=fivegram_weight
    
    
print(fivegram_weight_conet)

{'funeral_insurance:sunshine_coast:incentive_schemes:sydney:community': 0.005879790940766551, 'sunshine_coast:incentive_schemes:sydney:community:group': 0.004141739530602853, 'sunshine_coast:incentive_schemes:sydney:community:people': 0.0048320294523699955, 'sunshine_coast:incentive_schemes:sydney:community:indigenous_people': 0.004141739530602853, 'sunshine_coast:incentive_schemes:sydney:community:government': 0.004141739530602853, 'sunshine_coast:incentive_schemes:sydney:community:bank': 0.0048320294523699955, 'sydney_call:sunshine_coast:incentive_schemes:sydney:community': 0.036585365853658534, 'probe:sunshine_coast:incentive_schemes:sydney:community': 0.004573170731707317, 'death:probe:sunshine_coast:incentive_schemes:sydney': 0.009722222222222222, 'vulnerable_people:funeral_insurance:sunshine_coast:misconduct:indigenous_people': 0.0024250440917107582, 'vulnerable_people:funeral_insurance:sunshine_coast:misconduct:bank': 0.004188712522045855, 'vulnerable_people:funeral_insurance:su

In [67]:
# Make CSV of fivegram co-occurrence net

sourceNode_5g=[]
targetNode_5g=[]
edgeWeight_5g=[]

for key,val in fivegram_weight_conet.items():
    composite_topics=key.split(':')
    source=composite_topics[0]+':'+composite_topics[1]+':'+composite_topics[2]+':'+composite_topics[3]
    sourceNode_5g.append(source)
    targetNode_5g.append(composite_topics[4])
    edgeWeight_5g.append(val)
    
five_cooccur_df=pd.DataFrame()
five_cooccur_df['source']=sourceNode_5g
five_cooccur_df['target']=targetNode_5g
five_cooccur_df['weight']=edgeWeight_5g

five_cooccur_df.to_csv('50_topics_5gram_ooccurrence_net.csv',index=False,encoding='utf-8')

In [101]:
#import pprint

tstr='0.009*"financial" + 0.007*"insurance" + 0.006*"brexit" + 0.005*"funeral" + 0.005*"people" + 0.005*"acbf" + 0.004*"industry" + 0.004*"aboriginal" + 0.004*"services" + 0.004*"indigenous"'
ltop=[]
t1s=tstr.split('+')
for ctoken in t1s:
    ttoken=ctoken.split('*')
    itok = ttoken[1]
    token=itok.split('"')
    itop=token[1]
    #print(itop)
    ltop.append(np.str(itop))
    
print(ltop)
str1=np.str(ltop)
print(str1)
ndltop=list(set(ltop))
print(ndltop)
#tokens = tokenize_words(ltop)
#print(tokens)
#dtop = Dictionary([str1.split()])
#print("dictionary")
#print(dtop.keys)
#corpus = [id2word.doc2bow(text) for text in ltop]

    

['financial', 'insurance', 'brexit', 'funeral', 'people', 'acbf', 'industry', 'aboriginal', 'services', 'indigenous']
['financial', 'insurance', 'brexit', 'funeral', 'people', 'acbf', 'industry', 'aboriginal', 'services', 'indigenous']
['acbf', 'industry', 'indigenous', 'people', 'aboriginal', 'services', 'insurance', 'financial', 'brexit', 'funeral']
